In [1]:
def set_missing_priors_with_default(priors_dict, default_priors):
    

            
    for prior_name, value in default_priors.items():
        if prior_name not in priors_dict:
            priors_dict[prior_name] = value

In [2]:
import datetime
import logging
import pymc3 as pm
import pymc3 as pm
import theano
import theano.tensor as tt
import numpy as np
from pymc3 import Model


In [3]:

def delay_cases(cases, inp_len , out_len, diff_len, prior_mean_median=10, pr_sigma= 0.2, prior_median_width=0.3, ):
    with pm.Model() as Covid19:
        delay = pm.Normal(
            name= 'delay',
            mu = np.log(prior_mean_median),
            sigma = pr_sigma,
        )

    with pm.Model() as Covid19:
        pm.Deterministic("Delay", np.exp(delay))
    width = np.log(prior_median_width)
    delay_case = delay_lognormal(cases, inp_len, out_len, tt.exp(delay), tt.exp(width),diff_len)
    with pm.Model() as Covid19:
        pm.Deterministic('Delay_cases',delay_case)
    return delay_case


In [4]:
  
def delay_lognormal(cases, input_len, output_len, delay, width ,diff_len):
    delay_mat = delay_matrix(input_len, output_len, diff_len)
    delay_mat[delay_mat < 0.01] = 0.1
    delayed_arr = arr_delay(cases, delay, width, delay_mat)
    return delayed_arr
 

In [5]:
def delay_matrix(len1, len2, diff):
    size = max(len1,len2)
    mat=np.zeros((size,size))
    for i in range(size):
        diagonal = np.ones(size - i) * (diff + i)
        mat += np.diag(diagonal, i)
    for i in range(1, size):
        diagonal = np.ones(size - i) * (diff - i)
        mat += np.diag(diagonal, -i)
    
    return mat[:len1, :len2]
    
    

In [6]:

def arr_delay(cases, delay, width, delay_mat):
    
    mat = tt_lognormal(delay_mat, mu= np.log(delay),sigma = width)
    delayed_arr = tt.dot(cases, mat)
    
    return delayed_arr
    

In [7]:
       
def delay_sim_cases(new_I, lenI, len_out, delay, delay_diff):
    delay_mat= delay_matrix(lenI, len_out, delay_diff)
    
    infered_cases = interpolate(new_I, delay, delay_mat)
    return infered_cases
    
  

In [8]:

def interpolate(array, delay, delay_matrix):
    interp_matrix = tt.maximum(1 - tt.abs_(delay_matrix - delay), 0)
    interpolation = tt.dot(array, interp_matrix)
    return interpolation

In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


import os
import requests
import pymc3 as pm
import pandas as pd
import numpy as np
import theano
import theano.tensor as tt

from matplotlib import pyplot as plt
from matplotlib import dates as mdates
from matplotlib import ticker

from datetime import date
from datetime import datetime

from IPython.display import clear_output

import theano
import theano.tensor as tt

%config InlineBackend.figure_format = 'retina'

In [10]:
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"

In [11]:
import pandas as pd
import numpy as np
import datetime as DT

from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

from scipy import stats as sps
from scipy.interpolate import interp1d


from IPython.display import clear_output

In [12]:
today = DT.datetime.now().strftime("%Y%m%d")

# prepare data, to get daily cases and smoothing
def prepare_cases(cases, cutoff=25):

    smoothed = cases.rolling(7,    #7 days moving window for smoothing
        #win_type='gaussian',   #or comment whole line to have uniform
        min_periods=1,
        center=True).mean().round()
    
    idx_start = np.searchsorted(smoothed, cutoff)
    
    smoothed = smoothed.iloc[idx_start:]
    original = cases.loc[smoothed.index]
    
    return original, smoothed

In [13]:
path = url
states = pd.read_csv(url,
                    usecols=['location','date', 'new_cases'],
                     parse_dates=['date'],
                     index_col=['location', 'date'],
                     squeeze=True).sort_index()


In [14]:
country = ['Netherlands']
cases = states.xs(country)


In [15]:
cutoff= 25
smoothed = cases.rolling(7,    #7 days moving window for smoothing
        #win_type='gaussian',   #or comment whole line to have uniform
        min_periods=1,
        center=True).mean().round()
    
idx_start = np.searchsorted(smoothed, cutoff) 
    
smoothed = smoothed.iloc[idx_start:]
original = cases.loc[smoothed.index]

In [16]:
#convert into array for easier handling
original_array = original.values
smoothed_array = smoothed.values

# dates: what we have in real time are detected of cases, but they refer to infection happened several days ago
# comparing with Nowcasting procedures, this latancy is 8±1 days
dates = smoothed.index
dates_detection = date2num(smoothed.index.tolist())
dates_infection = smoothed.index - DT.timedelta(days=9)
dates_infection = date2num(dates_infection.tolist())

In [17]:
def tt_lognormal(x, mu, sigma):
    x = tt.clip(x,1e-12,1e12)
    sigma = tt.clip(sigma, 1e-12, 1e12)
    mu = tt.clip(mu, 1e-12, 1e12)
    distr = 1 / x * tt.exp(-((tt.log(x) - mu) ** 2) / (2 * sigma ** 2))
    return distr / (tt.sum(distr, axis=0) + 1e-12)

In [18]:
def lambda_t_with_sigmoids(
    change_points_list,
    sim_len,
    pr_median_lambda,
    pr_sigma_lambda_0=0.2,
    name_lambda_t="lambda_t", 
):
    
    lambda_list, tr_time_list, tr_len_list = _make_change_point_RVs(change_points_list, pr_median_lambda, pr_sigma_lambda_0)

    # Build the time-dependent spreading rate
    lambda_t_list = [
        lambda_list[0] * tt.ones(sim_len)
    ]  # model.sim_shape = (time, state)
    lambda_before = lambda_list[0]

    # Loop over all lambda values and there corresponding transient values
    for tr_time, tr_len, lambda_after in zip(
        tr_time_list, tr_len_list, lambda_list[1:]
    ):
        # Create the right shape for the time array
        t = np.arange(sim_len)
        tr_len = tr_len + 1e-5 
        
        # Applies standart sigmoid nonlinearity
        lambda_t = tt.nnet.sigmoid((t - tr_time) / tr_len * 4) * (
            lambda_after - lambda_before
        )  # tr_len*4 because the derivative of the sigmoid at zero is 1/4, we want to set it to 1/tr_len

        lambda_before = lambda_after
        lambda_t_list.append(lambda_t)

    # Sum up all lambda values from the list
    lambda_t_log = sum(lambda_t_list)

    # Create responding lambda_t pymc3 variable with given name (from parameters)
    with pm.Model(): 
        pm.Deterministic(name_lambda_t, tt.exp(lambda_t_log))

    return lambda_t_log


In [19]:

def _make_change_point_RVs(change_points_list, lambda_0, pr_sigma_lambda_0=1):
   
    def non_hierarchical():
        with pm.Model() as Covid19:
            lambda_0_log = pm.Normal(
                name="lambda_0_log", mu=np.log(lambda_0), sigma=pr_sigma_lambda_0
            )
        with pm.Model() as Covid19: 
            pm.Deterministic("lambda_0", tt.exp(lambda_0_log))
        lambda_log_list.append(lambda_0_log)

        # Create lambda_log list
        for i, cp in enumerate(change_points_list):
            pr_mean_lambda = np.log(cp["pr_median_lambda"])
            with pm.Model() as Covid19:
                lambda_cp_log = pm.Normal(
                    name=f"lambda_{i + 1}_log_",
                    mu=pr_mean_lambda,
                    sigma=cp["pr_sigma_lambda"],
                )
            with pm.Model() as Covid19:
                pm.Deterministic(f"lambda_{i + 1}", tt.exp(lambda_cp_log))
            lambda_log_list.append(lambda_cp_log)

           
        for i, cp in enumerate(change_points_list):
            dt_begin_transient = cp["pr_mean_date_transient"]
        
        
            prior_mean = 10
            with pm.Model() as Covid19:
                tr_time = pm.Normal(
                    name=f"transient_day_{i + 1}",
                    mu=prior_mean,
                    sigma=cp["pr_sigma_date_transient"],
                )
            tr_time_list.append(tr_time)
            
            
        for i, cp in enumerate(change_points_list):
            with pm.Model():
                tr_len_log = pm.Normal(
                    name= f"transient_len_{i + 1}_log_t",
                    mu=np.log(cp["pr_median_transient_len"]),
                    sigma=cp["pr_sigma_transient_len"],
                )
            with pm.Model() as Covid19:
                pm.Deterministic(f"transient_len_{i + 1}", tt.exp(tr_len_log))
                tr_len_list.append(tt.exp(tr_len_log))
            
    default_priors_change_points = dict(
        pr_median_lambda=lambda_0,
        pr_sigma_lambda=pr_sigma_lambda_0,
        pr_sigma_date_transient=2,
        pr_median_transient_len=4,
        pr_sigma_transient_len=0.5,
        pr_mean_date_transient=None,
        relative_to_previous=False,
        pr_factor_to_previous=1,
        )

    for cp_priors in change_points_list:
        set_missing_priors_with_default(cp_priors, default_priors_change_points)

    
    lambda_log_list = []
    tr_time_list = []
    tr_len_list = []
    
    non_hierarchical()


    

    return lambda_log_list, tr_time_list, tr_len_list



In [20]:

def _smooth_step_function(start_val, end_val, t_begin, t_end, t_total):
   
    t = np.arange(t_total)

    return (
         tt.clip((t - t_begin) / (t_end - t_begin), 0, 1) * (end_val - start_val)
        + start_val
    )

In [21]:
    import datetime
    prior_date_strong_dist_begin = datetime.datetime(2020, 3, 16)
    prior_date_contact_ban_begin = datetime.datetime(2020, 3, 23)

    change_points = [
        dict(
            pr_mean_date_transient=prior_date_strong_dist_begin,
            pr_sigma_date_transient=6,
            pr_median_lambda=1 / 8,
            pr_sigma_lambda=1,
        ),
        dict(
            pr_mean_date_transient=prior_date_contact_ban_begin,
            pr_sigma_date_transient=6,
            pr_median_lambda=1 / 8 / 2,
            pr_sigma_lambda=1,
        ),
    ]
    change_points = change_points[0:2]

In [22]:
change_points

[{'pr_mean_date_transient': datetime.datetime(2020, 3, 16, 0, 0),
  'pr_sigma_date_transient': 6,
  'pr_median_lambda': 0.125,
  'pr_sigma_lambda': 1},
 {'pr_mean_date_transient': datetime.datetime(2020, 3, 23, 0, 0),
  'pr_sigma_date_transient': 6,
  'pr_median_lambda': 0.0625,
  'pr_sigma_lambda': 1}]

In [23]:
def infec_prior():
    with pm.Model()  as Covid19:
        a = pm.Gamma(
        name = 'a', alpha =1, beta =1
        )
    with pm.Model() as Covid19:
        b = pm.Gamma(
        name = 'b', alpha =1, beta =1
        )
    return(a,b)

In [44]:

def SPEQIR(
    logb,  population,  gamma =0.34,  prior_E=50, prior_gamma = 1/8, pr_incubation= 5,
    pr_s_incubation= 2, sigma_incubation=0.4, pr_sigma_gamma=0.2):
    
    #Prior Exposed
    
    if isinstance(prior_E, tt.Variable):
        
        Exposed = prior_E
    else:
        with pm.Model() as Covid19:
            Exposed = pm.HalfCauchy(
                name ='exposed', beta=prior_E, shape=(16,138)
                )
    a,b = infec_prior()    
 
    # Prior Infectious
    with pm.Model() as Covid19:
        Incubated = pm.Gamma(
            name = 'incubated',  alpha = a, beta = b, shape= 138
        )

    S = population - Incubated - pm.math.sum(Exposed, axis=0) 

    lambda_t = tt.exp(logb)
    new_I = tt.zeros_like(Incubated)

    if pr_s_incubation is None:
        median_incubation = pr_incubation
    else:
        with pm.Model() as Covid19:
            median_incubation = pm.Normal(
                name = 'inc', mu=pr_incubation,
                sigma=pr_s_incubation,
            )
    
    

    # Choose transition rates (E to I) according to incubation period distribution
    
    x = np.arange(1, 16)[:, None]


    ro = tt_lognormal(x, tt.log(median_incubation), sigma_incubation)

    # Runs SEIR model:
    def next_day(
        lambda_t, S_t, nE1, nE2, nE3, nE4,nE5, nE6, nE7,nE8, nE9, nE10, nE11, nE12,nE13,nE14,nE15, I_t,_,  gamma,ro, N
    ):
        new_E_t = lambda_t / N * I_t * S_t
        S_t = S_t - new_E_t
        new_I_t = (
            ro[0] * nE1
            + ro[1] * nE2
            + ro[2] * nE3
            + ro[3] * nE4
            + ro[4] * nE5
            + ro[5] * nE6
            + ro[6] * nE7
            + ro[7] * nE8
            + ro[8] * nE9
            + ro[9] * nE10
            + ro[10]* nE11
            + ro[11] * nE12
            + ro[12] * nE13
            + ro[13] * nE14
            + ro[14] * nE15
            
        )
        I_t = I_t + new_I_t - (gamma)* I_t
        I_t = tt.clip(I_t, -1, N - 1)  # for stability
        S_t = tt.clip(S_t, -1, N)
        return S_t, new_E_t, I_t, new_I_t

    outputs, _ = theano.scan(
        fn=next_day,
        sequences=[lambda_t],
        outputs_info=[
            S,
            dict( initial = Exposed, taps=[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10,-11,-12,-13,-14,-15]),
            Incubated,
            new_I,
        ],
        non_sequences=[gamma, ro, population],
    )
    S_t, new_E_t, I_t, new_I_t = outputs
    with pm.Model()  as Covid19:
        pm.Deterministic('new_I_t', new_I_t)
        pm.Deterministic('S_t', S_t)
        pm.Deterministic('I_t', I_t)
        pm.Deterministic('new_E_t', new_E_t)

    return new_I_t




In [45]:
date_begin_data = datetime.datetime(2020, 3, 10)
date_end_data = datetime.datetime(2020, 3, 13)

In [46]:

        cases=smoothed,
        data_begin=date_begin_data,
        diff_len = 16,
        population= 17447213,


In [47]:

        lambda_t_log = lambda_t_with_sigmoids(
          change_points, len(smoothed) + 16 ,  pr_median_lambda=0.4
        )


In [48]:
sim_I = SPEQIR(lambda_t_log, 17447213)

In [63]:
def student_t_likelihood(
    data_obs,
    cases,
    data_len,
    pr_beta_sigma_obs=30,
    nu=4,
    offset_sigma=1,
):  
    
        
        
    with pm.Model() as Covid19:
        sigma_obs = pm.HalfCauchy(
        'sigmaobs', beta=pr_beta_sigma_obs, shape = 138,
    )
    
    with Covid19:
        I_t = pm.StudentT(
            name='name_student_t',
            nu=nu,
            mu=cases[:data_len],
            sigma=tt.abs_(cases[:data_len] + offset_sigma) ** 0.5
            * sigma_obs,  # offset and tt.abs to avoid nans
            observed =data_obs,
    )
    with Covid19:
        samples = pm.sample(draws=1000, tune=5000)
        

In [64]:
I_t = student_t_likelihood(cases,sim_I, len(cases))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


MissingInputError: Input 0 of the graph (indices start from 0), used to compute Elemwise{log,no_inplace}(inc), was not provided and not given a value. Use the Theano flag exception_verbosity='high', for more information on this error.

In [55]:
with Covid19:
    samples = pm.sample(draws=1000, tune=5000)

ValueError: The model does not contain any free variables.